## Text project

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [2]:
#Load dataset
def load_data():
    data = pd.read_csv('./data/Tweets.csv')
    return data

In [3]:
tweet_df = load_data()
tweet_df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &...",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)
tweet_df.info()

Dataset size: (14640, 15)
Columns are: Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
tweet_id                        14640 non-null int64
airline_sentiment               14640 non-null object
airline_sentiment_confidence    14640 non-null float64
negativereason                  9178 non-null object
negativereason_confidence       10522 non-null float64
airline                         14640 non-null object
airline_sentiment_gold          40 non-null object
name                            14640 non-null object
negativereason_gold             32 non-null object
retweet_count                   14640 non-

## Step 1 : take "text" column

In [7]:
tweet_df_minus = tweet_df['text']
tweet_df_minus[:5]

0                                                                    @VirginAmerica What @dhepburn said.
1                               @VirginAmerica plus you've added commercials to the experience... tacky.
2                                @VirginAmerica I didn't today... Must mean I need to take another trip!
3    @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &...
4                                                @VirginAmerica and it's a really big bad thing about it
Name: text, dtype: object


## Step 2: Data Preprocessing

We will perform the following steps: 

1. **Tokenization** : Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
2. Words that have fewer than 3 characters are removed.
3. All **stopwords** are removed.
4. Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
5. Words are **stemmed** - words are reduced to their root form.



In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lbenboudiaf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
Words_deleted_length = 3
Lemmatized = True
Stemmed = False

import pandas as pd
stemmer = SnowballStemmer("english")

'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    if(Lemmatized & Stemmed):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    if(Stemmed):
        return stemmer.stem(text)
    if(Lemmatized):
        return WordNetLemmatizer().lemmatize(text, pos='v')
    return text

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > Words_deleted_length:
            result.append(lemmatize_stemming(token))
            
    return result

In [11]:
'''We create two lists, one for pre-processed data and an other for not pre-processed data'''
processed_docs = []
nan_processed_docs = []
for doc in tweet_df_minus :
    nan_processed_docs.append(doc)
for doc in tweet_df_minus :
    processed_docs.append(preprocess(doc))
    

'''They should have the same size''' 
print('Size of not pre-processeded data : ', len(nan_processed_docs))
print('Size of pre-processeded data : ', len(processed_docs))
print('\nCheck data processing effect:')
print(nan_processed_docs[1])
print(processed_docs[1])

Size of not pre-processeded data :  14640
Size of pre-processeded data :  14640

Check data processing effect:
@VirginAmerica plus you've added commercials to the experience... tacky.
['virginamerica', 'plus', 'add', 'commercials', 'experience', 'tacky']


## Step 3: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass processed_docs to gensim.corpora.Dictionary() and call it 'dictionary'.


In [12]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)
print(dictionary)

Dictionary(9775 unique tokens: ['amypoehler', 'understand', 'friday', 'average', 'thanksm']...)


In [13]:
'''
Checking dictionary created
'''
count = 1
for k, v in dictionary.iteritems():
    print('Number of appearence:', k ,'Word:',v)
    count += 1
    if count > 10:
        break

Number of appearence: 4188 Word: amypoehler
Number of appearence: 498 Word: understand
Number of appearence: 509 Word: friday
Number of appearence: 2993 Word: average
Number of appearence: 9071 Word: thanksm
Number of appearence: 7324 Word: attendees
Number of appearence: 6072 Word: spill
Number of appearence: 7169 Word: livery
Number of appearence: 4677 Word: bijlxat
Number of appearence: 9508 Word: banana


In [14]:
'''
Create the Bag-of-words model for each document i.e. for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('The representation is a tuple : (word_id, word_frequency) for example : \n')
print(processed_docs[:2])
bow_corpus[:2]

The representation is a tuple : (word_id, word_frequency) for example : 

[['virginamerica', 'dhepburn', 'say'], ['virginamerica', 'plus', 'add', 'commercials', 'experience', 'tacky']]


[[(0, 1), (1, 1), (2, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]

In [15]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 2 ("virginamerica") appears 1 time.
Word 24 ("seat") appears 1 time.
Word 28 ("time") appears 1 time.
Word 88 ("available") appears 1 time.
Word 89 ("carriers") appears 1 time.
Word 90 ("fare") appears 1 time.
Word 91 ("select") appears 1 time.


## Step 4: Running LDA using Bag of Words
We are going for 10 topics in the document corpus.

We will be running LDA using all CPU cores to parallelize and speed up model training.

Some of the parameters we will be tweaking are:

1. **num_topics** is the number of requested latent topics to be extracted from the training corpus.
2. **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
3. **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
4. **alpha** and **beta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is 1/num_topics)

 **Alpha** is the per document topic distribution.
   - *High alpha* : Every document has a mixture of all topics(documents appear similar to each other).
   - *Low alpha* : Every document has a mixture of very few topics

  **bEta** is the per topic word distribution.
    - *High beta* : Each topic has a mixture of most words(topics appear similar to each other).
    - *Low eta* : Each topic has a mixture of few words.

5. **passes** is the number of training passes through the corpus. For example, if the training corpus has 50,000 documents, chunksize is 10,000, passes is 2, then online training is done in 10 updates:
- documents 0-9,999
- documents 10,000-19,999
- documents 20,000-29,999
- documents 30,000-39,999
- documents 40,000-49,999
- documents 0-9,999
- documents 10,000-19,999
- documents 20,000-29,999
- documents 30,000-39,999
- documents 40,000-49,999


In [18]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(corpus=bow_corpus, 
                                   num_topics = 8,
                                   chunksize=100,
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [19]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.026*"customers" + 0.021*"plan" + 0.020*"tonight" + 0.020*"appreciate" + 0.019*"airlines" + 0.019*"passengers" + 0.019*"sure" + 0.017*"gate" + 0.015*"money" + 0.015*"suppose"


Topic: 1 
Words: 0.137*"flight" + 0.054*"cancel" + 0.030*"hours" + 0.030*"wait" + 0.029*"hold" + 0.029*"jetblue" + 0.027*"delay" + 0.026*"flightled" + 0.019*"plane" + 0.017*"time"


Topic: 2 
Words: 0.079*"book" + 0.056*"ticket" + 0.038*"rebooked" + 0.027*"problems" + 0.025*"follow" + 0.023*"miles" + 0.018*"understand" + 0.016*"credit" + 0.016*"ask" + 0.016*"receive"


Topic: 3 
Words: 0.047*"change" + 0.037*"try" + 0.036*"online" + 0.028*"reservation" + 0.027*"check" + 0.021*"mins" + 0.018*"reservations" + 0.013*"charge" + 0.013*"stop" + 0.013*"tuesday"


Topic: 4 
Words: 0.253*"americanair" + 0.083*"usairways" + 0.033*"flight" + 0.033*"thank" + 0.025*"service" + 0.022*"help" + 0.020*"customer" + 0.013*"need" + 0.012*"http" + 0.011*"guy"


Topic: 5 
Words: 0.051*"luggage" + 0.041*"lose" + 0.03

## Step 5: Testing LDA model

In [20]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3425000748081727


Now that we have the baseline coherence score for the default LDA model, let’s perform a series of sensitivity tests to help determine the following model hyperparameters:

    Number of Topics (K)
    Dirichlet hyperparameter alpha: Document-Topic Density
    Dirichlet hyperparameter beta: Word-Topic Density
We’ll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two different validation corpus sets. We’ll use **C_v** as our choice of metric for performance comparison

We create function ***compute_coherence_values*** and iterate it over the range of topics, alpha, and beta parameter values

In [27]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import tqdm

grid = {} 
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(bow_corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(bow_corpus, num_of_docs*0.75), 
               bow_corpus]

corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pbar.close()

  4%|▍         | 23/540 [13:25<5:51:18, 40.77s/it]

lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

## Step 6: create T_e_i an input to have $t_{ei}$ after apply Doc2Vec 

In [107]:
list_topics=lda_model.show_topics(formatted=False)
list_topics 

topic = []
T_e_i = []
for tup in list_topics:
    topic = []
    for tup2 in tup[1]:
        topic.append(tup2[0])
    T_e_i.append(topic) 
    
T_e_i  

[['americanair',
  'http',
  'jetblue',
  'need',
  'usairways',
  'work',
  'check',
  'southwestair',
  'people',
  'good'],
 ['jetblue',
  'flight',
  'unite',
  'delay',
  'usairways',
  'southwestair',
  'plane',
  'thank',
  'americanair',
  'http'],
 ['americanair',
  'usairways',
  'service',
  'thank',
  'customer',
  'unite',
  'southwestair',
  'wait',
  'time',
  'luggage'],
 ['flight',
  'americanair',
  'cancel',
  'usairways',
  'southwestair',
  'help',
  'flightled',
  'hold',
  'book',
  'try']]

## Step 7 : Train Doc2Vec on non-pre-processed data "nan_processed_docs"


In [117]:
import smart_open
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def read(fname):
    for i, line in enumerate(f):
        tokens = fname[i]
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [121]:
train_nan_processed_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(nan_processed_docs)]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_nan_processed_docs)
model.train(train_nan_processed_docs, total_examples=len(nan_processed_docs), epochs=model.epochs)

## Step 8 : Apply Doc2Vec on each document in your data to form $d_{ei}$

In [124]:
Liste_D_e_i = []
Liste_of_n_docs = processed_docs
for i in range(len(Liste_of_n_docs)):
    vector = []
    vector = model.infer_vector(Liste_of_n_docs[i])
    Liste_D_e_i.append(vector) 
    
print(Liste_of_n_docs[:2])
print(Liste_D_e_i[:2])

[['virginamerica', 'dhepburn', 'say'], ['virginamerica', 'plus', 'add', 'commercials', 'experience', 'tacky']]
[array([-8.1121130e-03, -8.2422188e-03, -8.5009653e-03,  9.4700670e-03,
       -6.1424044e-03, -6.0291924e-03,  9.3967384e-03, -3.2938537e-03,
       -7.4124401e-03,  4.4365735e-03,  3.3982443e-03,  5.7742302e-03,
       -2.0059585e-03, -6.6704517e-03,  3.3299492e-03,  8.7660858e-03,
        3.0309090e-03,  9.5953336e-03,  2.2019406e-03, -3.9082011e-03,
       -5.5418705e-04, -7.8981556e-03,  1.1231005e-03, -7.9345554e-03,
        2.9577192e-05,  9.9902209e-03,  5.6541548e-03, -1.1336298e-03,
       -9.5684947e-03,  7.5632879e-03,  3.8973801e-03,  1.7543974e-04,
        3.7038026e-03,  6.9300961e-03, -5.4608597e-03,  5.8965799e-03,
       -8.2795974e-03, -2.2272721e-03,  1.6476009e-03,  4.3660291e-03,
        3.5117818e-03,  3.2109192e-03, -2.7814955e-03, -6.1880853e-03,
        8.8355839e-03, -2.6708855e-03,  3.3353553e-03,  7.7089434e-03,
        3.2863969e-03, -6.2256311e-0

## Step 9 : Apply Doc2Vec on your topics to form $t_{ei}$

In [126]:
Liste_T_e_i = []
for i in range(len(T_e_i)):
    vector = []
    vector = model.infer_vector(T_e_i[i])
    Liste_T_e_i.append(vector) 
Liste_T_e_i

[array([-1.6189611e-03, -5.7761609e-03,  8.8887429e-03,  7.1812181e-05,
         1.3372501e-03,  2.2232926e-03,  3.3773489e-03, -5.0686724e-03,
         2.3342411e-03,  9.5074857e-03, -5.6510912e-03,  4.9197194e-03,
        -6.3650357e-03,  8.8025462e-03, -4.6121841e-03, -4.7187209e-03,
        -6.0499255e-03,  8.9482125e-03,  4.5692851e-03, -1.5405561e-04,
        -4.2776135e-03,  7.6732994e-03, -9.7185532e-03, -1.5098313e-03,
         2.4043793e-04,  3.5099888e-03,  6.6660470e-03, -6.9918861e-03,
         7.2210683e-03, -9.8983021e-03,  1.9597057e-03, -1.4058575e-03,
         4.4692433e-03, -7.0016026e-03,  4.2176726e-03, -7.0681684e-03,
         9.9102091e-03, -7.9297964e-03,  6.6172373e-03,  9.6123023e-03,
         5.0199782e-03,  8.7301964e-03, -4.1522384e-03,  4.8934389e-03,
         2.2044850e-03,  9.0630125e-04,  3.2948253e-03, -8.0621773e-03,
         4.5056785e-03,  5.5571552e-03], dtype=float32),
 array([ 0.00275685, -0.00764542, -0.009705  ,  0.00741494, -0.00103935,
      

## Step 9 : Calculate the similarity between every pair $d_{ei}$ and $t_{ei}$
    Dont run this code bcz Liste_D_e_i length != Liste_T_e_i

In [ ]:
from scipy import spatial
similairty = []
for i in Liste_D_e_i:
    similairty[i] = spatial.distance.cosine(Liste_D_e_i[i], Liste_T_e_i[i])



## Step 10 : Graph

In [26]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
LDAvis_prepared

/home/lbenboudiaf/.local/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      27.790092        1       1 -0.318410 -0.129273
1      26.554451        1       2 -0.276953  0.083777
7       9.125051        1       3  0.032221  0.167865
0       8.692225        1       4  0.095855  0.131397
3       7.399105        1       5  0.048089  0.017618
6       7.114551        1       6  0.134770 -0.038359
2       6.854762        1       7  0.137886 -0.289615
5       6.469763        1       8  0.146542  0.056591, topic_info=     Category         Freq         Term        Total  loglift  logprob
717   Default  7920.000000  americanair  7920.000000  30.0000  30.0000
6631  Default  5144.000000       flight  5144.000000  29.0000  29.0000
6760  Default  2591.000000    usairways  2591.000000  28.0000  28.0000
8864  Default  1620.000000       cancel  1620.000000  27.0000  27.0000
440   Default   610.000000         book   610.000000  26.0000  26.0000
6371  Default   591.000000         seat   591.000000  25.0000  25.0000
153   Default   610.000000       change   610.000000  24.0000  24.0000
449   Default   910.000000        hours   910.000000  23.0000  23.0000
6094  Default   906.000000         wait   906.000000  22.0000  22.0000
936   Default   434.000000       ticket   434.000000  21.0000  21.0000
8919  Default   877.000000         hold   877.000000  20.0000  20.0000
656   Default   876.000000      jetblue   876.000000  19.0000  19.0000
6267  Default   610.000000          try   610.000000  18.0000  18.0000
8153  Default   819.000000        delay   819.000000  17.0000  17.0000
5475  Default   371.000000      luggage   371.000000  16.0000  16.0000
5025  Default   422.000000       number   422.000000  15.0000  15.0000
9236  Default   763.000000    flightled   763.000000  14.0000  14.0000
4098  Default   771.000000      service   771.000000  13.0000  13.0000
9704  Default   395.000000      airline   395.000000  12.0000  12.0000
6691  Default  1301.000000        thank  1301.000000  11.0000  11.0000
4948  Default   298.000000         lose   298.000000  10.0000  10.0000
3150  Default   637.000000     customer   637.000000   9.0000   9.0000
2166  Default   290.000000     rebooked   290.000000   8.0000   8.0000
1000  Default   297.000000       online   297.000000   7.0000   7.0000
650   Default   748.000000         http   748.000000   6.0000   6.0000
5813  Default   443.000000        check   443.000000   5.0000   5.0000
7539  Default   555.000000         gate   555.000000   4.0000   4.0000
3851  Default   454.000000          say   454.000000   3.0000   3.0000
2728  Default  1035.000000         help  1035.000000   2.0000   2.0000
6807  Default   249.000000       refund   249.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
2017   Topic8   149.790848         nice   150.723267   2.7318  -3.8836
7002   Topic8   132.934769         wasn   133.867477   2.7310  -4.0030
3832   Topic8   129.228104        extra   130.160889   2.7308  -4.0312
739    Topic8   119.156479        tweet   120.088715   2.7302  -4.1124
1462   Topic8   114.466248         poor   115.398727   2.7299  -4.1525
1496   Topic8   110.777794         pick   111.710136   2.7296  -4.1853
993    Topic8   105.785408      airways   106.717819   2.7293  -4.2314
7722   Topic8    90.728539        claim    91.660538   2.7278  -4.3849
3437   Topic8    83.911072    charlotte    84.843719   2.7270  -4.4631
7420   Topic8    76.942245        sleep    77.874680   2.7260  -4.5498
9604   Topic8    72.276634      believe    73.209198   2.7252  -4.6123
126    Topic8    67.150017  information    68.082451   2.7242  -4.6859
3159   Topic8    64.362152         real    65.294418   2.7236  -4.7283
3865   Topic8    61.537659         yeah    62.470062   2.7230  -4.7732
8116   Topic8    57.453072        human    58.385193   2.7219  -4.8419
5328   Topic8    57.356892         away    58.289310   2.7219  -4.8435
4263   To